In [66]:
import numpy as np
from datasets import load_dataset

dataset = load_dataset("brainer/pill_identification_data")
dataset

DatasetDict({
    train: Dataset({
        features: ['품목일련번호', '품목명', '업소일련번호', '업소명', '성상', '큰제품이미지', '표시앞', '표시뒤', '의약품제형', '색상앞', '색상뒤', '분할선앞', '분할선뒤', '크기장축', '크기단축', '크기두께', '이미지생성일자(약학정보원)', '분류번호', '분류명', '전문일반구분', '품목허가일자', '제형코드명', '표기내용앞', '표기내용뒤', '표기이미지앞', '표기이미지뒤', '표기코드앞', '표기코드뒤', '변경일자', '사업자번호'],
        num_rows: 25330
    })
})

In [67]:
# 품목일련번호 2개 이상인 row visualize
dataset_tmp = dataset["train"].to_pandas()
dataset_tmp = dataset_tmp.groupby('품목일련번호').count()
dataset_tmp = dataset_tmp[dataset_tmp['품목명'] > 1]
duplicate_lit = list(dataset_tmp.index)

dataset_tmp = dataset["train"].to_pandas()
dataset_tmp = dataset_tmp[dataset_tmp['품목일련번호'].isin(duplicate_lit)]
dataset_tmp

,품목일련번호,품목명,업소일련번호,업소명,성상,큰제품이미지,표시앞,표시뒤,의약품제형,색상앞,...,품목허가일자,제형코드명,표기내용앞,표기내용뒤,표기이미지앞,표기이미지뒤,표기코드앞,표기코드뒤,변경일자,사업자번호
194,200906540,로나센정2밀리그램(블로난세린)/로나센정4밀리그램(블로난세린),19630001,부광약품(주),흰색의원형정제,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,마크,L2,원형,하양,...,20090826,나정,川|≡,-,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,-,r0008,-,20220608,1188100450
411,200906540,로나센정2밀리그램(블로난세린)/로나센정4밀리그램(블로난세린),19630001,부광약품(주),흰색의원형정제,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,마크,L4,원형,하양,...,20090826,나정,川|≡,-,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,-,r0008,-,20220608,1188100450
1833,200209641,라믹탈정25밀리그램|50밀리그램|100밀리그램(라모트리진),20161652,(주)글락소스미스클라인,엷은황갈색의정방형정제,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,GSEE1,50,사각형,노랑,...,20021230,나정,-,-,-,-,-,-,20211108,1068156683
1834,200209641,라믹탈정25밀리그램|50밀리그램|100밀리그램(라모트리진),20161652,(주)글락소스미스클라인,엷은황갈색의정방형정제,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,GSEE5,100,사각형,노랑,...,20021230,나정,-,-,-,-,-,-,20211108,1068156683
1835,200209641,라믹탈정25밀리그램|50밀리그램|100밀리그램(라모트리진),20161652,(주)글락소스미스클라인,엷은황갈색의정방형정제,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,GSEC7,25,사각형,노랑,...,20021230,나정,-,-,-,-,-,-,20211108,1068156683
6027,200201478,치옥타시드에이취알정200|600밀리그램(티옥트산),19630001,부광약품(주),200밀리그람:밝은녹색의양쪽이볼록한원형필름코팅정600밀리그람:황녹색의양쪽이볼록한장방...,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,HR,-,장방형,연두,...,20020311,필름코팅정,-,-,-,-,-,-,20191202,1188100450
6036,200202793,핀테정(에포니디핀염산염),19910005,(주)녹십자,백색또는미황백색의원형필름코팅정이다,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,FT40,-,원형,하양,...,20021126,필름코팅정,-,-,-,-,-,-,20130913,3038117108
7011,200401818,부스론정5밀리그램|10밀리그램(부스피론염산염),19630001,부광약품(주),양면볼록한장방형의백색정제,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,B분할선K,10,장방형,하양,...,20040214,나정,-,-,-,-,-,-,20220517,1188100450
7012,200401818,부스론정5밀리그램|10밀리그램(부스피론염산염),19630001,부광약품(주),양면볼록한장방형의백색정제,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,BK,5,장방형,하양,...,20040214,나정,-,-,-,-,-,-,20220517,1188100450
7226,200202793,핀테정(에포니디핀염산염),19910005,(주)녹십자,백색또는미황백색의원형필름코팅정이다,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,FT20,-,원형,하양,...,20021126,필름코팅정,-,-,-,-,-,-,20130913,3038117108


In [2]:
dataset = dataset["train"]
dataset = dataset.select_columns(['품목일련번호', '크기장축', '크기단축', '크기두께'])


# Drop 품목명 == '보넥실캅셀<소브레롤>'
def drop_not_float(example):
    """
    크기장축, 크기단축, 크기두께가 float으로 변환 가능한지 확인
    :param example: 
    :return: 
    """
    try:
        float(example["크기장축"])
        float(example["크기단축"])
        float(example["크기두께"])
        return True
    except ValueError:
        return False


dataset = dataset.filter(drop_not_float)
dataset

Dataset({
    features: ['품목일련번호', '크기장축', '크기단축', '크기두께'],
    num_rows: 25221
})

In [3]:
# dtype str to int
dataset = dataset.map(
    lambda example: {
        "크기장축": np.asarray(example["크기장축"]).astype(np.float64),
        "크기단축": np.asarray(example["크기단축"]).astype(np.float64),
        "크기두께": np.asarray(example["크기두께"]).astype(np.float64),
    },
    num_proc=7,
    batched=True,
    batch_size=1000,
    keep_in_memory=True,
)

Map (num_proc=7):   0%|          | 0/25221 [00:00<?, ? examples/s]

In [4]:
# Node: 품목일련번호: 식품의약품안전처 약품식별정보및 상세정보 보기 원료약품 및 분량, 효능효과, 사용상의주의사항,재심사, RMP, 보험, 기타정보, 생산실적
# Edge: 크기장축, 크기단축, 크기두께를 고려한 상대적 부피

# 부피
dataset = dataset.add_column(name="크기", column=list(np.asarray(dataset["크기장축"]) * np.asarray(dataset["크기단축"])))

In [5]:
dataset.column_names

['품목일련번호', '크기장축', '크기단축', '크기두께', '크기']

In [6]:
from tqdm.notebook import tqdm
from networkx import Graph

G = Graph()

# dataset = dataset.shard(num_shards=100, index=0).to_pandas()

# '크기' 열의 값들을 리스트로 추출
size_list = dataset['크기']

# 가정: '품목일련번호' 열의 값들이 리스트로 주어짐
item_numbers = dataset['품목일련번호']

# 노드 추가
for item in tqdm(item_numbers):
    G.add_node(item)

edge_count = {item: 0 for item in item_numbers}

# 엣지 추가
for i, item_i in tqdm(enumerate(item_numbers), total=len(item_numbers)):
    for j, item_j in enumerate(item_numbers):
        if item_i != item_j and edge_count[item_i] < 3:
            # 여기서 size_list를 사용하여 가중치를 계산합니다.
            # size_list가 0인 경우를 처리하기 위한 조건문을 추가할 수 있습니다.
            weight = size_list[i] / size_list[j] if size_list[j] != 0 else 0
            G.add_edge(item_i, item_j, weight=weight)
            edge_count[item_i] += 1  # 에지 수 업데이트

  0%|          | 0/25221 [00:00<?, ?it/s]

  0%|          | 0/25221 [00:00<?, ?it/s]

In [11]:
from pyvis.network import Network

# pyvis의 Network 객체 생성
net = Network(notebook=True)
net = net.from_nx(G)
net.write_html("size-graph.html")
# net.show("size-graph.html")

AttributeError: 'NoneType' object has no attribute 'write_html'

In [10]:
import networkx as nx
from datasets import Dataset

edge_ds = Dataset.from_pandas(nx.to_pandas_edgelist(G).astype({"source": "str", "target": "str"}))
edge_ds.push_to_hub("brainer/pill_identification_graph", config_name="size-graph-edges")
edge_ds

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/76 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

Dataset({
    features: ['source', 'target', 'width', 'weight'],
    num_rows: 75609
})

In [9]:
import pandas as pd

node_ds = Dataset.from_pandas(pd.DataFrame(G.nodes, columns=["id"]).astype({"id": "str"}))
node_ds.push_to_hub("brainer/pill_identification_graph", config_name="size-graph-nodes")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/26 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/2.33k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/brainer/pill_identification_graph/commit/e9dde6a0b2614a6d84c7b7beacf49783a6aa2fce', commit_message='Upload dataset', commit_description='', oid='e9dde6a0b2614a6d84c7b7beacf49783a6aa2fce', pr_url=None, pr_revision=None, pr_num=None)